In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np
from keras.datasets import mnist
# seed for reproducing same results
seed = 785
np.random.seed(seed)

C:\Users\Midhilesh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset = np.loadtxt('ocr.csv', delimiter=',')
X = dataset[:,0:784]
Y = dataset[:,0]

In [3]:
import pandas as pd
dataset1=pd.read_csv('mnist.csv')
print(dataset1.head())

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         

In [4]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.50)

In [5]:
c=dataset1.columns.values.tolist()
X1=dataset1[c[1:]]
Y1=dataset1[c[0]]
Y1=Y1.replace([0,1,2,3,4,5,6,7,8,9],[26,27,28,29,30,31,32,33,34,35])
X1=np.array(X1)
Y1=np.array(Y1)
x_train1,x_test1,y_train1,y_test1=train_test_split(X1,Y1,test_size=0.50)


In [6]:
train_feature=np.vstack((x_train,x_train1))
train_label=np.concatenate((y_train,y_train1))
test_feature=np.vstack((x_test,x_test1))
test_label=np.concatenate((y_test,y_test1))

<class 'numpy.ndarray'>


In [7]:
train_feature = train_feature.reshape(train_feature.shape[0], 28, 28, 1)
test_feature = test_feature.reshape(test_feature.shape[0], 28, 28, 1)

train_feature = train_feature / 255
test_feature = test_feature / 255
# one hot encode outputs
train_label = np_utils.to_categorical(train_label)
test_label = np_utils.to_categorical(test_label)

In [8]:
print(train_feature.shape)

(207225, 28, 28, 1)


In [9]:
from keras import backend as K
K.set_image_dim_ordering('tf')

In [10]:
num_classes =36
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=(28, 28, 1), activation='relu'))
model.add(Conv2D(32, (2, 2), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 27, 27, 32)        160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        4128      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        16448     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
__________

In [12]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_feature,train_label,validation_data=(test_feature,test_label), epochs=5, batch_size=200, verbose=1)


Train on 207225 samples, validate on 207226 samples
Epoch 1/5
207225/207225 [==============================] - 478s 2ms/step - loss: 0.4176 - acc: 0.8836 - val_loss: 0.1354 - val_acc: 0.9614
Epoch 2/5
207225/207225 [==============================] - 473s 2ms/step - loss: 0.1561 - acc: 0.9551 - val_loss: 0.1018 - val_acc: 0.9708
Epoch 3/5
207225/207225 [==============================] - 471s 2ms/step - loss: 0.1218 - acc: 0.9643 - val_loss: 0.0878 - val_acc: 0.9752
Epoch 4/5
207225/207225 [==============================] - 484s 2ms/step - loss: 0.1012 - acc: 0.9704 - val_loss: 0.0764 - val_acc: 0.9783
Epoch 5/5
207225/207225 [==============================] - 485s 2ms/step - loss: 0.0899 - acc: 0.9733 - val_loss: 0.0695 - val_acc: 0.9805


In [13]:
scores = model.evaluate(test_feature,test_label, verbose=1)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

207226/207226 [==============================] - 116s 557us/step
CNN Error: 1.95%


In [14]:
model.save('alphanum2.model')

In [1]:
from keras.models import load_model
model=load_model('alphanum2.model')

C:\Users\Midhilesh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [16]:
a=x_test1[7].reshape(1,28,28,1)
classes=model.predict_classes(a)
print(classes)
print(y_test1[7])

[26]
26


In [4]:

import cv2
import numpy as np
img=cv2.imread('q.png',0)

x1=np.array(img)
x1=x1/255
x1=x1.reshape(1,28,28,1)
classes = model.predict_classes(x1)
print(classes)

[16]
